In [19]:
from __future__ import print_function

import warnings
warnings.simplefilter(action='ignore')

import keras
from keras.models import load_model
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

from PIL import Image
from keras.preprocessing.image import img_to_array

REDUCED_IMAGE_SIZE = 1024

# Deploying a model to AWS as a Flask API

## Step 1 - Train, evaluate and save the model

### First, let's do some preprocessing

In [20]:
batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(f'input_shape: {input_shape}')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(f'x_train shape: {x_train.shape}')
print(f'# train samples: {x_train.shape[0]}')
print(f'# test samples: {x_test.shape[0]}')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

input_shape: (28, 28, 1)
x_train shape: (60000, 28, 28, 1)
# train samples: 60000
# test samples: 10000


### Now we have to define the model

In [21]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 94s 2ms/step - loss: 0.2680 - acc: 0.9181 - val_loss: 0.0531 - val_acc: 0.9822


In [22]:
# Save the model
model.save('my_model.h5')

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('M:')
print(y_test[0])
print(x_test[0].shape)
x = x_test[0].reshape(1, 28, 28, 1)
out = model.predict(x)
print(out[0])
print(np.argmax(out[0]))

Test loss: 0.05308448419899214
Test accuracy: 0.9822
M:
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(28, 28, 1)
[6.3977637e-09 1.1692205e-08 1.3490408e-07 3.2882971e-07 2.3187701e-09
 1.5504812e-09 1.6863844e-11 9.9999797e-01 1.1532186e-08 1.5675189e-06]
7


### We now have trained and saved the model to model_weights.h5

## Step 2 - Test the model on a real image

In [23]:
image_filename = "img_35.jpg"

img = Image.open(image_filename)
# img = img.resize((REDUCED_IMAGE_SIZE, REDUCED_IMAGE_SIZE))
test_image = img_to_array(img)
test_image = test_image.reshape(1, 28, 28, 1)

In [24]:
model.predict(test_image)

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

**It predicts 6!! as we expected**

## Step 3 - Try to load model from h5 file

In [25]:
model = load_model('my_model.h5')

## Step 3 - Build Flask API - Look to flask app.py file for code on this